In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import time
import ollama
import json
from utils import load_json, save_json

In [4]:
cases = []
for root, dirs, files in os.walk('../bigger_study_sample/'):
    for file in files:
        if file.endswith('.json'):
            cases.append(os.path.join(root, file))

In [5]:
cases

['../bigger_study_sample/001-85449.json',
 '../bigger_study_sample/001-170054.json',
 '../bigger_study_sample/001-145342.json',
 '../bigger_study_sample/001-147040.json',
 '../bigger_study_sample/001-57899.json',
 '../bigger_study_sample/001-57676.json',
 '../bigger_study_sample/001-111142.json',
 '../bigger_study_sample/001-57430.json']

In [6]:
import re

def get_last_question(text):
    # Regex pattern to match questions, considering multiline questions
    pattern = r'Question:\s*(.*?)(?=\n[A-Z]|\Z)'

    # Find all matches with the multiline flag enabled
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Get the last match
    last_question = matches[-1] if matches else None

    return last_question


def check_for_stop(text):
    # Regex pattern to match questions, considering multiline questions
    pattern = r'Stop:\s*(.*?)(?=\n[A-Z]|\Z)'

    # Find all matches with the multiline flag enabled
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Get the last match
    stop = matches[-1] if matches else None

    return stop

def check_for_final_answer(text):
    # Regex pattern to match questions, considering multiline questions
    pattern = r'Final Answer:\s*(.*?)(?=\n[A-Z]|\Z)'

    # Find all matches with the multiline flag enabled
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Get the last match
    stop = matches[-1] if matches else None

    return stop


In [7]:
from agent_senior import SeniorAgent
from agent_junior import JuniorAgent

In [8]:
s_agent = SeniorAgent()
j_agent = JuniorAgent()

In [9]:
c4 = load_json('../bigger_study_sample/001-57899.json')

In [10]:
def run_conversation(s_agent, j_agent, case):
    round = 20
    all_conversation = ""

    for i in range(round):
        print(f"---------------------round{i+1}-------------------------")
        all_conversation += f"---------------------round{i+1}-------------------------\n"
        response = s_agent.ask_question()
        question = get_last_question(response)
        print(f"Q: {question}")
        stop = check_for_stop(response)
        final_answer = check_for_final_answer(response)
        if (stop is not None) or (final_answer is not None):
            print(f"Stop: {stop}, final_answer: {final_answer}")
            print(response)
            all_conversation += f"\n{response}"
            break

        answer = j_agent.answer_question(case['facts'], case['conclusion'], question)
        s_agent.append_to_cache(answer)
        
        print(f"Senior Agent -> {question}")
        print(f"Junior Agent -> {answer}")

        all_conversation += f"Senior Agent -> {question}\n\n Junior Agent -> {answer}\n\n"
        time.sleep(20)
    return all_conversation

# C4

In [11]:
s_agent = SeniorAgent()
j_agent = JuniorAgent()

all_conversation = run_conversation(s_agent, j_agent, c4)

output_name = c4['itemid']+ '_output.json'
output_dict = {'cache': s_agent.cache}
save_json(output_name, output_dict)

---------------------round1-------------------------
Q: What are the specific facts of the legal case that I should consider for this task?
Senior Agent -> What are the specific facts of the legal case that I should consider for this task?
Junior Agent -> Answer: The specific facts of the legal case that I should consider for this task are:

* The applicant, Mr. Anthony Boner, was convicted of armed robbery and sentenced to eight years imprisonment.
* He was denied legal aid for his appeal, despite having substantial grounds for taking the appeal.
* His solicitors and counsel had declined to continue to act for him because they believed the grounds of appeal were not substantial.
* The Scottish Legal Aid Board refused his application for legal aid for the appeal, citing that there were no substantial grounds for making the appeal.
* Mr. Boner proceeded with his appeal without legal representation, and the appeal court dismissed his appeal.
* The appeal court judges took particular care

KeyboardInterrupt: 

In [15]:
print(all_conversation)

---------------------round1-------------------------
Senior Agent: What are the specific facts of the case that I should consider for this task?

 Junior Agent: Answer: The specific facts of the case that I should consider for this task are:

* The applicant, Mr. Anthony Boner, was convicted of armed robbery and sentenced to eight years imprisonment.
* He was denied legal aid for his appeal, despite having substantial grounds for appeal.
* His solicitors and counsel declined to continue acting for him because they believed the grounds of appeal were not substantial.
* The Scottish Legal Aid Board refused his application for legal aid, citing that there were no substantial grounds for making the appeal.
* Mr. Boner proceeded with his appeal without legal representation, and the appeal court dismissed his appeal.
* The appeal court judges took particular care to ensure that Mr. Boner was treated with courtesy and consideration, and counsel for the Crown was present but did not actively p

In [16]:
def save_txt_file(file_name, content):
    with open(file_name, 'w') as f:
        f.write(content)

In [17]:
save_txt_file(c4['itemid']+ '_output.txt', all_conversation)